In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
from tqdm import tqdm

In [4]:
from geopy.geocoders import Nominatim
# geolocalização
geolocator = Nominatim(user_agent="myApp", timeout=5)

In [5]:
with open('data/houses.pkl', 'rb') as f:
    obj = pickle.load(f)

In [24]:
obj['https://www.vivareal.com.br/imovel/apartamento-2-quartos-santo-amaro-zona-sul-sao-paulo-com-garagem-80m2-aluguel-RS4400-id-2567996191/'].__dict__

{'type': None,
 'address': 'Avenida Doutor Mário Vilas Boas Rodrigues, 175 - Santo Amaro, São Paulo - SP',
 'url': 'https://www.vivareal.com.br/imovel/apartamento-2-quartos-santo-amaro-zona-sul-sao-paulo-com-garagem-80m2-aluguel-RS4400-id-2567996191/',
 'price': {'rent': 3900,
  'Condomínio': ' R$ 746 ',
  'Preço c/ condomínio': 'R$ 4.646',
  'IPTU': ' R$ 225 '},
 'total_price': None,
 'area': 80,
 'bedrooms': 2,
 'bathrooms': [21],
 'parking': 1,
 'amenities': ['Piscina',
  'Churrasqueira',
  'Condomínio fechado',
  'Academia',
  'Espaço gourmet',
  'Ar-condicionado',
  'Jardim',
  'Área de serviço',
  'Playground',
  'Quadra poliesportiva',
  'Cozinha',
  'Gerador elétrico',
  'Segurança 24h',
  'Salão de jogos',
  'Sala de massagem',
  'Spa'],
 'description': None,
 'transport': None,
 'publisher_name': None,
 'code': None,
 'title': '  Apartamento com 2 Quartos e 2 banheiros para Alugar, 80 m² por R$ 3.900/Mês  '}

In [26]:
df = pd.DataFrame(obj.values())

In [27]:
df

,type,address,url,price,total_price,area,bedrooms,bathrooms,parking,amenities,description,transport,publisher_name,code,title
0,None,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 3900, 'Condomínio': ' R$ 746 ', 'Preç...",None,80.0,2.0,[21],1.0,"[Piscina, Churrasqueira, Condomínio fechado, A...",None,None,None,None,Apartamento com 2 Quartos e 2 banheiros para...
1,None,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,"{'rent': 900, 'Condomínio': ' Não informado ',...",None,40.0,1.0,[1],NaN,[],None,None,None,None,Sobrado com 1 Quarto e 1 banheiro para Aluga...
2,None,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 2500, 'Condomínio': ' R$ 726 ', 'Preç...",None,38.0,1.0,[1],1.0,"[Churrasqueira, Academia, Piscina]",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...
3,None,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 1700, 'Condomínio': ' R$ 350 ', 'Preç...",None,27.0,1.0,[11],NaN,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...
4,None,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 3500, 'Condomínio': ' R$ 706 ', 'Preç...",None,45.0,1.0,[11],1.0,"[Piscina, Churrasqueira, Elevador, Condomínio ...",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,None,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,"{'rent': 3700, 'Condomínio': ' R$ 950 ', 'Preç...",None,50.0,1.0,[11],1.0,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",None,None,None,None,Apartamento com 1 Quarto e 1 banheiro para A...
1822,None,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,"{'rent': 4500, 'Condomínio': ' R$ 3.500 ', 'Pr...",None,100.0,NaN,[3],3.0,[],None,None,None,None,"Sala Comercial e 3 banheiros para Alugar, 10..."
1823,None,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,"{'rent': 3900, 'Condomínio': ' R$ 870 ', 'Preç...",None,87.0,NaN,[1],2.0,[],None,None,None,None,"Sala Comercial e 1 banheiro para Alugar, 87 ..."
1824,None,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,"{'rent': 1970, 'Condomínio': ' R$ 720 ', 'Preç...",None,60.0,NaN,[1],NaN,"[Ar-condicionado, Elevador, Condomínio fechado...",None,None,None,None,"Ponto Comercial e 1 banheiro para Alugar, 60..."


Ajustando o valor do imóvel

In [28]:
df = pd.concat([df, pd.DataFrame(df.price.to_dict().values(), columns=['rent', 'Condomínio', 'Preço c/ condomínio','IPTU'])], axis=1)

Ajustando o banheiro, removendo a lista

In [29]:
df.bathrooms = df.bathrooms.map(lambda x: int(x[0]) if isinstance(x, list) and (not not x) else 0)

In [30]:
df.iloc[0:3].address.apply(geolocator.geocode).apply(lambda x: pd.Series([x.latitude, x.longitude], index=['location_lat', 'location_long']))

,location_lat,location_long
0,-23.650586,-46.718644
1,-23.723508,-46.693460
2,-21.132831,-51.102942


In [31]:
df.drop(columns=['total_price', 'area', 'description', 'transport', 'publisher_name', 'code', 'type'], inplace=True)

In [32]:
df['Condomínio'] = df['Condomínio'].str.replace('R$ ', '', regex=False)

In [33]:
df['Condomínio'] = df['Condomínio'].str.strip()

In [34]:
df['Condomínio'] = df['Condomínio'].replace({'Não informado': 0, 'Preço c/ condomínio':0, 'IPTU':'0'})

In [35]:
df['Condomínio'] = df['Condomínio'].str.replace('.', '', regex=False)

In [36]:
df['Condomínio'] = df['Condomínio'].fillna(0)

In [37]:
df['Condomínio'] = df['Condomínio'].astype(int)

In [38]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].replace({'R$ ':''}, regex=False).fillna(0)

In [39]:
df['Preço c/ condomínio'] = df['Preço c/ condomínio'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).fillna(0).astype(int)

In [40]:
df['IPTU'] = df['IPTU'].str.replace('R$ ','', regex=False).str.replace('.', '', regex=False).replace({' Não informado ':0}, regex=False).fillna(0).astype(int)

In [41]:
df.drop(columns=['price'], inplace=True)

Endereço e amenities

Considerar os bairos.

In [44]:
df.shape

(1826, 11)

In [48]:
df.sample(frac=0.05)#pega 5% do dataframe

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,IPTU
1449,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0
844,"Vila Leonor, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,2.0,"[Piscina, Mobiliado, Churrasqueira, Elevador, ...",Apartamento com 2 Quartos e 2 banheiros para...,1850.0,780,2630,174
940,"Avenida Ramalho Ortigão, 356 - Vila Gumercindo...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,2.0,"[Piscina, Mobiliado, Elevador, Condomínio fech...",Apartamento com 2 Quartos e 2 banheiros para...,4500.0,920,5420,349
950,"Rua Kansas - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sobrado-3-q...,3.0,31,1.0,[],"Sobrado com 3 Quartos e 3 banheiros à Venda,...",2000000.0,0,0,0
547,"Rua das Fiandeiras - Vila Olímpia, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Piscina, Churrasqueira, Elevador, Varanda, Ac...",Apartamento com 1 Quarto e 1 banheiro para A...,2700.0,860,3560,170
...,...,...,...,...,...,...,...,...,...,...,...
474,"Rua Castrinópolis - Vila Santa Maria, São Paul...",https://www.vivareal.com.br/imovel/casa-1-quar...,1.0,1,NaN,[],"Casa com 1 Quarto e 1 banheiro para Alugar, ...",1000.0,0,0,0
489,"Rua Carneiro Leão, 385 - Brás, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,2.0,1,1.0,"[Churrasqueira, Elevador, Condomínio fechado, ...",Apartamento com 2 Quartos e 1 banheiro para ...,1280.0,370,1650,80
335,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0
324,None,https://www.vivareal.com.br/imovel/apartamento...,NaN,0,NaN,[],None,NaN,0,0,0


In [99]:
#organizar as amenities em ordem alfabética
df.amenities = df.amenities.apply(lambda x: sorted(x))

In [103]:
# identificando todas as amenities
amenities = []
df.amenities.apply(lambda x : amenities.extend(x))
amenities = set(sorted(amenities))

In [106]:
df_amenities = pd.DataFrame(df.amenities.apply(lambda x: {_:1 for _ in x}).to_list())

In [110]:
df_amenities.fillna(0, inplace=True)

In [111]:
pd.concat([df, df_amenities], axis=1)

,address,url,bedrooms,bathrooms,parking,amenities,title,rent,Condomínio,Preço c/ condomínio,...,Drywall,Muro de vidro,Piso elevado,Poço artesiano,Muro e grade,Espaço teen,Pista de skate,Children care,Vista para lago,Centro de estética
0,"Avenida Doutor Mário Vilas Boas Rodrigues, 175...",https://www.vivareal.com.br/imovel/apartamento...,2.0,21,1.0,"[Academia, Ar-condicionado, Churrasqueira, Con...",Apartamento com 2 Quartos e 2 banheiros para...,3900.0,746,4646,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Rua Fulgêncio Vertello - Cidade Dutra, São Pau...",https://www.vivareal.com.br/imovel/sobrado-1-q...,1.0,1,NaN,[],Sobrado com 1 Quarto e 1 banheiro para Aluga...,900.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Mirandópolis, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,1,1.0,"[Academia, Churrasqueira, Piscina]",Apartamento com 1 Quarto e 1 banheiro para A...,2500.0,726,3226,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Rua dos Estudantes, 357 - Sé, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,NaN,"[Academia, Aceita animais, Bicicletário, Churr...",Apartamento com 1 Quarto e 1 banheiro para A...,1700.0,350,2050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alameda Itu, 78 - Jardins, São Paulo - SP",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Academia, Aceita animais, Acesso para deficie...",Apartamento com 1 Quarto e 1 banheiro para A...,3500.0,706,4206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,"Rua Henri Dunant, 1066 - Chácara Santo Antônio...",https://www.vivareal.com.br/imovel/apartamento...,1.0,11,1.0,"[Academia, Aceita animais, Acesso para deficie...",Apartamento com 1 Quarto e 1 banheiro para A...,3700.0,950,4650,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1822,"Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,3,3.0,[],"Sala Comercial e 3 banheiros para Alugar, 10...",4500.0,3500,8000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1823,"Rua Arandu, 205 - Brooklin, São Paulo - SP",https://www.vivareal.com.br/imovel/sala-comerc...,NaN,1,2.0,[],"Sala Comercial e 1 banheiro para Alugar, 87 ...",3900.0,870,4770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1824,"Avenida Paulista - Bela Vista, São Paulo - SP",https://www.vivareal.com.br/imovel/ponto-comer...,NaN,1,NaN,"[Ar-condicionado, Condomínio fechado, Elevador...","Ponto Comercial e 1 banheiro para Alugar, 60...",1970.0,720,2690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
df.to_excel('data/dados.xlsx', index=False)

## Identificar a geolocalização de cada endereço

In [113]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="MyApp")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [114]:
geolocator = Nominatim(user_agent="myApp")

In [117]:
df.iloc[0].address

'Avenida Doutor Mário Vilas Boas Rodrigues, 175 - Santo Amaro, São Paulo - SP'

In [135]:
g=geocode('Rua General Chagas Santos - Saúde, São Paulo - SP', addressdetails =True)
g

Location(Rua General Chagas Santos, Vila do Bosque, Saúde, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Geográfica Intermediária de São Paulo, São Paulo, Região Sudeste, 04138-030, Brasil, (-23.6157408, -46.6272546, 0.0))

In [130]:
tqdm.pandas()

In [139]:
df['geocode'] = None

In [144]:
v

0       Avenida Doutor Mário Vilas Boas Rodrigues, 175...
1       Rua Fulgêncio Vertello - Cidade Dutra, São Pau...
2                            Mirandópolis, São Paulo - SP
3            Rua dos Estudantes, 357 - Sé, São Paulo - SP
4               Alameda Itu, 78 - Jardins, São Paulo - SP
                              ...                        
1821    Rua Henri Dunant, 1066 - Chácara Santo Antônio...
1822    Rua Jesuíno Arruda, 769 - Itaim Bibi, São Paul...
1823           Rua Arandu, 205 - Brooklin, São Paulo - SP
1824        Avenida Paulista - Bela Vista, São Paulo - SP
1825    Rua Frei Duarte Jorge de Mendonça, 126 - Vila ...
Name: address, Length: 1826, dtype: object

In [155]:
geo_dic = {}

In [156]:
%%time
for i, v in tqdm(df.iterrows(), total=df.shape[0]):
#     df.loc[i, 'geocode'] = 
    geo_dic[i] = geocode(v['address'], addressdetails =True)

  6%|████▍                                                                          | 102/1826 [01:41<28:36,  1.00it/s]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "

  8%|██████▎                                                                        | 146/1826 [02:47<27:21,  1.02it/s]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Americanópolis, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File 

 16%|████████████▋                                                                  | 292/1826 [05:40<26:02,  1.02s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Centro, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py", line 472, in wrap_socket
    cnx.do_handshake()
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1894, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1607, in _raise_ssl_error
    raise WantReadError()
OpenSSL.SSL.WantReadError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 344, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Alexandre\anaconda3\lib\site

RateLimiter swallowed an error after 2 retries. Called with (*('Centro, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Alexandre\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Alexandre\

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Centro, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py", line 472, in wrap_socket
    cnx.do_handshake()
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1894, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1607, in _raise_ssl_error
    raise WantReadError()
OpenSSL.SSL.WantReadError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 344, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 843, in _validate_conn
    conn.connect()
  File "C:\Users\Alexandre\anacond

 18%|██████████████▎                                                                | 331/1826 [07:11<26:44,  1.07s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rua Napoleão de Barros, 874 - Vila Clementino, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
OSError: [WinError 10051] Uma operação de soquete foi tentada em uma rede inacessível

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Avenida Senador Casimiro da Rocha - Mirandópolis, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Alexandre\anaconda3\lib\socket.py", line 704, in readinto
    return self._

 21%|████████████████▌                                                              | 382/1826 [08:42<32:55,  1.37s/it]


KeyboardInterrupt: 

In [175]:
pd.DataFrame(geo_dic.items()).drop(columns=0).rename({1:'geocode'}, axis=1)

,1
0,"(Avenida Doutor Mário Vilas Boas Rodrigues, Sa..."
1,"(Rua Fulgêncio Vertello, Cidade Dutra, São Pau..."
2,"(Mirandópolis, Região Imediata de Andradina, R..."
3,"(Rua dos Estudantes, Itaim Paulista, São Paulo..."
4,None
...,...
377,"(None, Torino, Piemonte, Italia, (44.933143, 7..."
378,"(237, Rua Bela Cintra, Consolação, São Paulo, ..."
379,"(Rua Orfanato, Vila Prudente, São Paulo, Regiã..."
380,"(Praça Barão de Itaqui, Vila Gomes Cardim, Tat..."


In [167]:
pd.concat([df, pd.DataFrame(geo_dic.values())], axis=1)

TypeError: 'NoneType' object is not iterable

In [138]:
%%time
df['geocode'] = df['address'].progress_apply(geocode, addressdetails =True)#.progress_apply(lambda location: (location.latitude, location.longitude))
# df['coord'] = df['address'].progress_apply(geocode).progress_apply(lambda location: (location.latitude, location.longitude))
# df.head()

  6%|████▍                                                                          | 103/1826 [01:44<28:43,  1.00s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "

RateLimiter swallowed an error after 2 retries. Called with (*('Avenida Jônia, 326 - Jardim Brasil, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Alexandre\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)

RateLimiter caught an error, retrying (1/2 tries). Called with (*('Americanópolis, São Paulo - SP',), **{'addressdetails': True}).
Traceback (most recent call last):
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "C:\Users\Alexandre\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Alexandre\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Alexandre\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users

 10%|███████▋                                                                       | 179/1826 [03:48<35:05,  1.28s/it]


KeyboardInterrupt: 

In [134]:
df.iloc[5]

address                Rua General Chagas Santos - Saúde, São Paulo - SP
url                    https://www.vivareal.com.br/imovel/apartamento...
bedrooms                                                             1.0
bathrooms                                                              1
parking                                                              1.0
amenities              [Ar-condicionado, Churrasqueira, Cozinha, Elev...
title                    Apartamento com 1 Quarto e 1 banheiro para A...
rent                                                              2300.0
Condomínio                                                           506
Preço c/ condomínio                                                 2806
IPTU                                                                 183
Name: 5, dtype: object